# CS336 Assignment 2 (systems): Systems and Parallelism

## 1.1 Profiling and Benchmarking

### Problem (benchmarking_script): 4 points

Write a script to perform basic end-to-end benchmarking of the forward and backward passes in your model. Specifically, your script should support the following:

- Given hyperparameters (e.g., number of layers), initialize a model.
- Generate a random batch of data.
- Run $w$ warm-up steps (before you start measuring time), then time the execution of $n$ steps (either only forward, or both forward and backward passes, depending on an argument). For timing, you can use the Python `timeit` module (e.g., either using the `timeit` function, or using `timeit.default_timer()`, which gives you the system’s highest resolution clock, thus a better default for benchmarking than `time.time()`).
- Call `torch.cuda.synchronize()` after each step.

**Deliverable**: A script that will initialize a basics Transformer model with the given hyperparameters, create a random batch of data, and time forward and backward passes.

---

<div style="background-color: rgb(196, 196, 196); padding: 10px; color: #333;">

[benchmark.py](./scripts/benchmark.py)

</div>

---

Time the forward and backward passes for the model sizes described in $\S 1.1.2$. Use $5$ warmup steps and compute the average and standard deviation of timings over 10 measurement steps.

How long does a forward pass take? How about a backward pass? Do you see high variability across measurements, or is the standard deviation small?

**Deliverable**: A 1-2 sentence response with your timings.

---

<div style="background-color: rgb(196, 196, 196); padding: 10px; color: #333;">

**Small model**: A forward pass takes ~25ms, and a backward pass takes ~69ms.

**Medium model**: A forward pass takes ~79ms, and a backward pass takes ~202ms.

Larger models result in a "CUDA out of memory" error.

The measurements are very stable.

</div>

---

One caveat of benchmarking is not performing the warm-up steps. Repeat your analysis without the warm-up steps. How does this affect your results? Why do you think this happens? Also try to run the script with 1 or 2 warm-up steps. Why might the result still be different?

**Deliverable**: A 2-3 sentence response.

---

<div style="background-color: rgb(196, 196, 196); padding: 10px; color: #333;">

Without warm-up (warm-up=0), the first run is much slower (70.95 ms vs. 25–26 ms later) and has high variance. 

This is due to one-time costs like CUDA context init, memory allocation, and JIT compilation. 

Just 1–2 warm-up runs greatly reduce the time, showing most overhead is paid early.

</div>

---

### Problem (nsys_profile): 5 points

Profile your forward pass, backward pass, and optimizer step using `nsys` with each of the model sizes described in Table 1 and context lengths of `128`, `256`, `512` and `1024` (you may run out of memory with some of these context lengths for the larger models, in which case just note it in your report).

small_fwd
![small_fwd](./data/imgs/small_fwd.png)

ctx1024_fwd
![ctx1024_fwd](./data/imgs/ctx1024_fwd.png)


(a) What is the total time spent on your forward pass? Does it match what we had measured before with the Python standard library?

**Deliverable**: A 1-2 sentence response.

---

<div style="background-color: rgb(196, 196, 196); padding: 10px; color: #333;">

The total time spent on a forward pass, measured by the Python script during profiling, was ~14.09 ms for the small_fwd model and ~23.33 ms for the ctx1024_fwd model. 

This timing is consistent with the duration of the corresponding NVTX time ranges observed in the Nsight Systems profiler's timeline view.

</div>

---

(b) What CUDA kernel takes the most cumulative GPU time during the forward pass? How many times is this kernel invoked during a single forward pass of your model? Is it the same kernel that takes the most runtime when you do both forward and backward passes? (Hint: look at the “CUDA GPU Kernel Summary” under “Stats Systems View”, and filter using NVTX ranges to identify which parts of the model are responsible for which kernels.)

**Deliverable**: A 1-2 sentence response.

---

<div style="background-color: rgb(196, 196, 196); padding: 10px; color: #333;">

The CUDA kernel that takes the most cumulative GPU time is for **GEMM**, general matrix multiplication (ampere_sgemm_128x64_tn, ampere_fp16_s1688gemm_fp16_128x128_ldg8_f2f_stages_32x1_tn).

For the small_fwd model, this kernel was invoked 42 times during a single forward pass (calculated from 630 total invocations over 15 steps). This same GEMM kernel type also dominates the runtime when performing both forward and backward passes.

</div>

---

(c) Although the vast majority of `FLOPs` take place in matrix multiplications, you will notice that several other kernels still take a non-trivial amount of the overall runtime. What other kernels besides matrix multiplies do you see accounting for non-trivial CUDA runtime in the forward pass?

**Deliverable**: A 1-2 sentence response.

---

<div style="background-color: rgb(196, 196, 196); padding: 10px; color: #333;">

Other non-trivial CUDA runtime comes mainly from **element-wise operations** kernels(e.g., `at::native::elementwise_kernel`, `vectorized_elementwise_kernel`) used in activations, residual adds, and LayerNorm.ctions, residual connections, and parts of the Layer Normalization computation.

</div>

---

(d) Profile running one complete training step with your implementation of AdamW (i.e., the forward pass, computing the loss and running a backward pass, and finally an optimizer step, as you’d do during training). How does the fraction of time spent on matrix multiplication change, compared to doing inference (forward pass only)? How about other kernels?

**Deliverable**: A 1-2 sentence response.

---

<div style="background-color: rgb(196, 196, 196); padding: 10px; color: #333;">

Compared to forward-only, a full training step (forward + backward + optimizer) sees GEMM’s share of time drop, while **element-wise** kernels take a larger share. 

This is because AdamW is memory-bound and made of element-wise ops, adding step time without adding GEMMs, thus diluting GEMM’s percentage.

</div>

---

small_fwd_annot
![small_fwd_annot](./data/imgs/small_fwd_annot.png)

small_fwbw_annot
![small_fwbw_annot](./data//imgs/small_fwbw_annot.png)

(e) Compare the runtime of the softmax operation versus the matrix multiplication operations within the self-attention layer of your model during a forward pass. How does the difference in runtimes compare to the difference in FLOPs?

**Deliverable**: A 1-2 sentence response.

---

<div style="background-color: rgb(196, 196, 196); padding: 10px; color: #333;">

In self-attention, GEMM takes much longer than Softmax, but the gap is far smaller than their FLOP difference. 

This is because GEMM is **compute-bound** with high arithmetic intensity, fully utilizing the GPU, while Softmax is **memory-bound**, limited by data access, making its runtime disproportionately high relative to its low FLOPs.

</div>

---

### Problem (mixed_precision_accumulation): 1 point

Run the following code and commment on the (accuracy of the) results.

**Deliverable**: A 2-3 sentence response.

In [1]:
import torch

s = torch.tensor(0,dtype=torch.float32)
for i in range(1000):
	s += torch.tensor(0.01,dtype=torch.float32)
print(s)
s = torch.tensor(0,dtype=torch.float16)
for i in range(1000):
	s += torch.tensor(0.01,dtype=torch.float16)
print(s)
s = torch.tensor(0,dtype=torch.float32)
for i in range(1000):
	s += torch.tensor(0.01,dtype=torch.float16)
print(s)
s = torch.tensor(0,dtype=torch.float32)
for i in range(1000):
	x = torch.tensor(0.01,dtype=torch.float16)
	s += x.type(torch.float32)
print(s)

tensor(10.0001)
tensor(9.9531, dtype=torch.float16)
tensor(10.0021)
tensor(10.0021)


---

<div style="background-color: rgb(196, 196, 196); padding: 10px; color: #333;">

The first result (`tensor(10.0001)`) shows accurate FP32 accumulation. 

The second (`tensor(9.9531, dtype=torch.float16)`) shows significant precision loss from FP16 rounding errors over many iterations. 

Results 3 and 4 (`tensor(10.0021)`) demonstrate that FP32 accumulation—whether FP16 values are auto-promoted (case 3) or explicitly converted (case 4)—yields identical accuracy, as PyTorch promotes types automatically in mixed-precision addition.

</div>

---

### Problem (benchmarking_mixed_precision): 2 points

Consider the following model:

```python
class ToyModel(nn.Module):
    def __init__(self, in_features: int, out_features: int):
        super().__init__()
        self.fc1 = nn.Linear(in_features, 10, bias=False)
        self.ln = nn.LayerNorm(10)
        self.fc2 = nn.Linear(10, out_features, bias=False)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.ln(x)
        x = self.fc2(x)
        return x
```

Suppose we are training the model on a GPU and that the model parameters are originally in FP32. We’d like to use autocasting mixed precision with FP16. What are the data types of:

  * the model parameters within the autocast context,
  * the output of the first feed-forward layer (`ToyModel.fc1`),
  * the output of layer norm (`ToyModel.ln`),
  * the model’s predicted logits,
  * the loss,
  * and the model’s gradients?

**Deliverable**: The data types for each of the components listed above.

---

<div style="background-color: rgb(196, 196, 196); padding: 10px; color: #333;">



</div>

---

You should have seen that FP16 mixed precision autocasting treats the layer normalization layer differently than the feed-forward layers. What parts of layer normalization are sensitive to mixed precision? If we use BF16 instead of FP16, do we still need to treat layer normalization differently? Why or why not?

**Deliverable**: A 2-3 sentence response.

---

<div style="background-color: rgb(196, 196, 196); padding: 10px; color: #333;">



</div>

---

Modify your benchmarking script to optionally run the model using mixed precision with BF16. Time the forward and backward passes with and without mixed-precision for each language model size described in $\\S 1.1.2$. Compare the results of using full vs. mixed precision, and comment on any trends as model size changes. You may find the `nullcontext` no-op context manager to be useful.

**Deliverable**: A 2-3 sentence response with your timings and commentary.

---

<div style="background-color: rgb(196, 196, 196); padding: 10px; color: #333;">



</div>

---


### Problem (memory_profiling): 4 points

Profile your forward pass, backward pass, and optimizer step of the 2.7B model from Table 1 with context lengths of 128, 256 and 512.

-----

<span style="background-color: #29B6F6; color: black">

</span>

-----

(a) Add an option to your profiling script to run your model through the memory profiler. It may be helpful to reuse some of your previous infrastructure (e.g., to activate mixed-precision, load specific model sizes, etc). Then, run your script to get a memory profile of the 2.7B model when either doing inference only (just forward pass) or a full training step. How do your memory timelines look like? Can you tell which stage is running based on the peaks you see?

**Deliverable**: Two images of the “Active memory timeline” of a 2.7B model, from the `memory_viz` tool: one for the forward pass, and one for running a full training step (forward and backward passes, then optimizer step), and a 2-3 sentence response.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

(b) What is the peak memory usage of each context length when doing a forward pass? What about when doing a full training step?

**Deliverable**: A table with two numbers per context length.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

(c) Find the peak memory usage of the 2.7B model when using mixed-precision, for both a forward pass and a full optimizer step. Does mixed-precision significantly affect memory usage?

**Deliverable**: A 2-3 sentence response.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

(d) Consider the 2.7B model. At our reference hyperparameters, what is the size of a tensor of activations in the Transformer residual stream, in single-precision? Give this size in MB (i.e., divide the number of bytes by $1024^2$).

**Deliverable**: A 1-2 sentence response with your derivation.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

(e) Now look closely at the “Active Memory Timeline” from `pytorch.org/memory_viz` of a memory snapshot of the 2.7B model doing a forward pass. When you reduce the “Detail” level, the tool hides the smallest allocations to the corresponding level (e.g., putting “Detail” at 10% only shows the 10% largest allocations). What is the size of the largest allocations shown? Looking through the stack trace, can you tell where those allocations come from?

**Deliverable**: A 1-2 sentence response.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

## 1.2 Optimizing Attention with FlashAttention-2

### Problem (pytorch_attention): 2 points

Benchmark your attention implementation at different scales. Write a script that will:

1. Fix the batch size to 8 and don’t use multihead attention (i.e. remove the head dimension).
2. Iterate through the cartesian product of [16, 32, 64, 128] for the head embedding dimension $d_{model}$, and [256, 1024, 4096, 8192, 16384] for the sequence length.
3. Create random inputs $Q, K, V$ for the appropriate size.
4. Time 100 forward passes through attention using the inputs.
5. Measure how much memory is in use before the backward pass starts, and time 100 backward passes.
(f) Make sure to warm up, and to call `torch.cuda.synchronize()` after each forward/backward pass.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

Report the timings (or out-of-memory errors) you get for these configurations. At what size do you get out-of-memory errors? Do the accounting for the memory usage of attention in one of the smallest configurations you find that runs out of memory (you can use the equations for memory usage of Transformers from Assignment 1). How does the memory saved for backward change with the sequence length? What would you do to eliminate this memory cost?

**Deliverable**: A table with your timings, your working out for the memory usage, and a 1-2 paragraph response.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

## 1.3 Benchmarking JIT-Compiled Attention

### Problem (torch_compile): 2 points

(a) Extend your attention benchmarking script to include a compiled version of your PyTorch implementation of attention, and compare its performance to the uncompiled version with the same configuration as the `pytorch_attention` problem above.

**Deliverable**: A table comparing your forward and backward pass timings for your compiled attention module with the uncompiled version from the `pytorch_attention` problem above.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

(b) Now, compile your entire Transformer model in your end-to-end benchmarking script. How does the performance of the forward pass change? What about the combined forward and backward passes and optimizer steps?

**Deliverable**: A table comparing your vanilla and compiled Transformer model.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

### Problem (flash_forward): 15 points

(a) Write a pure PyTorch (no Triton) `autograd.Function` that implements the FlashAttention-2 forward pass. This will be a lot slower than the regular PyTorch implementation, but will help you debug your Triton kernel.

Your implementation should take input $Q, K$, and $V$ as well as a flag `is_causal` and produce the output $O$ and the `logsumexp` value $L$. You can ignore the `is_causal` flag for this task. The `autograd.Function` forward should then use save $L, Q, K, V, O$ for the backward pass and return $O$. Remember that the implementation of the forward method of `autograd.Function` always takes the context as its first parameter. Any `autograd.Function` class needs to implement a backward method, but for now you can make it just raise `NotImplementedError`. If you need something to compare against, you can implement Equation 4 to 6 and 12 in PyTorch and compare your outputs.

The interface is then `def forward(ctx, Q, K, V, is_causal=False)`. Determine your own tile sizes, but make sure they are at least of size $16 \\times 16$. We will always test your code with dimensions that are clean powers of 2 and at least 16, so you don’t need to worry about out-of-bounds accesses.

**Deliverable**: A `torch.autograd.Function` subclass that implements FlashAttention-2 in the forward pass. To test your code, implement `[adapters.get_flashattention_autograd_function_pytorch]`. Then, run the test with `uv run pytest -k test_flash_forward_pass_pytorch` and make sure your implementation passes it.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

(b) Write a Triton kernel for the forward pass of FlashAttention-2 following Algorithm 1. Then, write another subclass of `torch.autograd.Function` that calls this (fused) kernel in the forward pass, instead of computing the result in PyTorch. A few problem-specific tips:

  * To debug, we suggest comparing the results of each Triton operation you perform with the tiled PyTorch implementation you wrote in part (a).
  * Your launch grid should be set as `(Tq, batch_size)`, meaning each Triton program instance will load only elements from a single batch index, and only read/write to a single query tile of $Q, O$, and $L$.
  * The kernel should only have a single loop, which will iterate key tiles $1 \\le j \\le T_k$.
  * Advance block pointers at the end of the loop.
  * Use the function declaration below (using the block pointer we give you, you should be able to infer the setup of the rest of the pointers):

<!-- end list -->

```python
@triton.jit
def flash_fwd_kernel(
    Q_ptr, K_ptr, V_ptr,
    O_ptr, L_ptr,
    stride_qb, stride_qq, stride_qd,
    stride_kb, stride_kk, stride_kd,
    stride_vb, stride_vk, stride_vd,
    stride_ob, stride_oq, stride_od,
    stride_lb, stride_lq,
    N_QUERIES, N_KEYS,
    scale,
    D: tl.constexpr,
    Q_TILE_SIZE: tl.constexpr,
    K_TILE_SIZE: tl.constexpr,
):
    # Program indices
    query_tile_index = tl.program_id(0)
    batch_index = tl.program_id(1)

    # Offset each pointer with the corresponding batch index
    # multiplied with the batch stride for each tensor
    Q_block_ptr = tl.make_block_ptr(
        Q_ptr + batch_index * stride_qb,
        shape=(N_QUERIES, D),
        strides=(stride_qq, stride_qd),
        offsets=(query_tile_index * Q_TILE_SIZE, 0),
        block_shape=(Q_TILE_SIZE, D),
        order=(1, 0),
    )

    ...
```

where `scale` is $\\frac{1}{\\sqrt{d}}$ and `Q_TILE_SIZE` and `K_TILE_SIZE` are $B_q$ and $B_k$ respectively. You can tune these later.

These additional guidelines may help you avoid precision issues:

  * The on chip buffers ($O_i, l, m$) should have dtype `tl.float32`. If you’re accumulating into an output buffer, use the `acc` argument (`acc = tl.dot(..., acc=acc)`).
  * Cast $\\tilde{P}^{(j)}_i$ to the dtype of $V^{(j)}$ before multiplying them, and cast $O_i$ to the appropriate dtype before writing it to global memory. Casting is done with `tensor.to`. You can get the dtype of a tensor with `tensor.dtype`, and the dtype of a block pointer/pointer with `*_block_ptr.type.element_ty`.

**Deliverable**: A `torch.autograd.Function` subclass that implements FlashAttention-2 in the forward pass using your Triton kernel. Implement `[adapters.get_flash_autograd_function_triton]`. Then, run the test with `uv run pytest -k test_flash_forward_pass_triton` and make sure your implementation passes it.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

(c) Add a flag as the last argument to your `autograd.Function` implementation for causal masking. This should be a boolean flag that when set to True enables an index comparison for causal masking. Your Triton kernel should have a corresponding additional parameter `is_causal: tl.constexpr` (this is a required type annotation). In Triton, construct appropriate index vectors for queries and keys, and compare them to form a square mask of size $B_q \\times B_k$. For elements that are masked out, add the constant value of $-1e6$ to the corresponding elements of the attention score matrix $S^{(j)}_i$. Make sure to save the mask flag for backward using `ctx.is_causal = is_causal`.

**Deliverable**: An additional flag for your `torch.autograd.Function` subclass that implements the FlashAttention-2 forward pass with causal masking using your Triton kernel. Make sure that the flag is optional with default `False` so the previous tests still pass.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

### Problem (flash_backward): 5 points

Implement the backward pass for your FlashAttention-2 `autograd.Function` using PyTorch (not Triton) and `torch.compile`. Your implementation should take the $Q, K, V, O, dO$, and $L$ tensors as output, and return $dQ, dK$ and $dV$. Remember to compute and use the $D$ vector. You may follow along the computations of Equations 13 to 19.

**Deliverable**: To test your implementation, run `uv run pytest -k test_flash_backward`.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

### Problem (flash_benchmarking): 5 points

Write a benchmarking script using `triton.testing.do_bench` that compares the performance of your (partially) Triton implementation of FlashAttention-2 forward and backward passes with a regular PyTorch implementation (i.e., not using FlashAttention). Specifically, you will report a table that includes latencies for forward, backward, and the end-to-end forward-backward pass, for both your Triton and PyTorch implementations. Randomly generate any necessary inputs before you start benchmarking, and run the benchmark on a single H100. Always use batch size 1 and causal masking. Sweep over the cartesian product of sequence lengths of various powers of 2 from 128 up to 65536, embedding dimension sizes of various powers of 2 from 16 up to size 128, and precisions of `torch.bfloat16` and `torch.float32`. You will likely need to adjust tile sizes depending on the input sizes.

**Deliverable**: A table of results comparing your implementation of FlashAttention-2 with the PyTorch implementation, using the settings above and reporting forward, backward, and end-to-end latencies.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----


---

## 2.1 Single-Node Distributed Communication in PyTorch

### Problem (distributed_communication_single_node): 5 points

Write a script to benchmark the runtime of the all-reduce operation in the single-node multi-process setup. The example code above may provide a reasonable starting point. Experiment with varying the following settings:

  * Backend + device type: Gloo + CPU, NCCL + GPU.
  * all-reduce data size: float32 data tensors ranging over 1MB, 10MB, 100MB, 1GB.
  * Number of processes: 2, 4, or 6 processes.
  * Resource requirements: Up to 6 GPUs. Each benchmarking run should take less than 5 minutes.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

**Deliverable**: Plot(s) and/or table(s) comparing the various settings, with 2-3 sentences of commentary about your results and thoughts about how the various factors interact.

## 2.2 A Naïve Implementation of Distributed Data Parallel Training

### Problem (naive_ddp): 5 points

**Deliverable**: Write a script to naively perform distributed data parallel training by all-reducing individual parameter gradients after the backward pass. To verify the correctness of your DDP implementation, use it to train a small toy model on randomly-generated data and verify that its weights match the results from single-process training.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

### Problem (naive_ddp_benchmarking): 3 points

In this naïve DDP implementation, parameters are individually all-reduced across ranks after each backward pass. To better understand the overhead of data parallel training, create a script to benchmark your previously-implemented language model when trained with this naïve implementation of DDP. Measure the total time per training step and the proportion of time spent on communicating gradients. Collect measurements in the single-node setting (1 node $\times$ 2 GPUs) for the XL model size described in $\S 1.1.2$.


**Deliverable**: A description of your benchmarking setup, along with the measured time per training iteration and time spent communicating gradients for each setting.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

## 2.3 Improving Upon the Minimal DDP Implementation

### Problem (minimal_ddp_flat_benchmarking): 2 points

Modify your minimal DDP implementation to communicate a tensor with flattened gradients from all parameters. Compare its performance with the minimal DDP implementation that issues an all-reduce for each parameter tensor under the previously-used conditions (1 node $\times$ 2 GPUs, XL model size as described in $\S 1.1.2$).

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

**Deliverable**: The measured time per training iteration and time spent communicating gradients under distributed data parallel training with a single batched all-reduce call. 1-2 sentences comparing the results when batching vs. individually communicating gradients.

### Problem (ddp_overlap_individual_parameters): 5 points

Implement a Python class to handle distributed data parallel training. The class should wrap an arbitrary PyTorch `nn.Module` and take care of broadcasting the weights before training (so all ranks have the same initial parameters) and issuing communication calls for gradient averaging. We recommend the following public interface:

```python
def __init__(self, module: torch.nn.Module):
```

Given an instantiated PyTorch `nn.Module` to be parallelized, construct a DDP container that will handle gradient synchronization across ranks.

```python
def forward(self, *inputs, **kwargs):
```

Calls the wrapped module’s `forward()` method with the provided positional and keyword arguments.

```python
def finish_gradient_synchronization(self):
```

When called, wait for asynchronous communication calls to be queued on GPU.

To use this class to perform distributed training, we’ll pass it a module to wrap, and then add a call to `finish_gradient_synchronization()` before we run `optimizer.step()` to ensure that the optimizer step, an operation that depends on the gradients, may be queued:

```python
model = ToyModel().to(device)
ddp_model = DDP(model)
for _ in range(train_steps):
    x, y = get_batch()
    logits = ddp_model(x)
    loss = loss_fn(logits, y)
    loss.backward()
    ddp_model.finish_gradient_synchronization()
    optimizer.step()
```

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

**Deliverable**: Implement a container class to handle distributed data parallel training. This class should overlap gradient communication and the computation of the backward pass. To test your DDP class, first implement the adapters `[adapters.get_ddp_individual_parameters]` and `[adapters.ddp_individual_parameters_on_after_backward]` (the latter is optional, depending on your implementation you may not need it). Then, to execute the tests, run `uv run pytest tests/test_ddp_individual_parameters.py`. We recommend running the tests multiple times (e.g., 5) to ensure that it passes reliably.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

### Problem (ddp_overlap_individual_parameters_benchmarking): 1 point

(a) Benchmark the performance of your DDP implementation when overlapping backward pass computation with communication of individual parameter gradients. Compare its performance with our previously-studied settings (the minimal DDP implementation that either issues an all-reduce for each parameter tensor, or a single all-reduce on the concatenation of all parameter tensors) with the same setup: 1 node, 2 GPUs, and the XL model size described in $\\S 1.1.2$.

**Deliverable**: The measured time per training iteration when overlapping the backward pass with communication of individual parameter gradients, with 1-2 sentences comparing the results.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

(b) Instrument your benchmarking code (using the 1 node, 2 GPUs, XL model size setup) with the Nsight profiler, comparing between the initial DDP implementation and this DDP implementation that overlaps backward computation and communication. Visually compare the two traces, and provide a profiler screenshot demonstrating that one implementation overlaps compute with communication while the other doesn’t.

**Deliverable**: 2 screenshots (one from the initial DDP implementation, and another from this DDP implementation that overlaps compute with communication) that visually show that communication is or isn’t overlapped with the backward pass.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

### Problem (ddp_overlap_bucketed): 8 points

Implement a Python class to handle distributed data parallel training, using gradient bucketing to improve communication efficiency. The class should wrap an arbitrary input PyTorch `nn.Module` and take care of broadcasting the weights before training (so all ranks have the same initial parameters) and issuing bucketed communication calls for gradient averaging. We recommend the following interface:

```python
def __init__(self, module: torch.nn.Module, bucket_size_mb: float):
```

Given an instantiated PyTorch `nn.Module` to be parallelized, construct a DDP container that will handle gradient synchronization across ranks. Gradient synchronization should be bucketed, with each bucket holding at most `bucket_size_mb` of parameters.

```python
def forward(self, *inputs, **kwargs):
```

Calls the wrapped module’s `forward()` method with the provided positional and keyword arguments.

```python
def finish_gradient_synchronization(self):
```

When called, wait for asynchronous communication calls to be queued on GPU.

Beyond the addition of a `bucket_size_mb` initialization parameter, this public interface matches the interface of our previous DDP implementation that individually communicated each parameter. We suggest allocating parameters to buckets using the reverse order of `model.parameters()`, since the gradients will become ready in approximately that order during the backward pass.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

**Deliverable**: Implement a container class to handle distributed data parallel training. This class should overlap gradient communication and the computation of the backward pass. Gradient communication should be bucketed, to reduce the total number of communication calls. To test your implementation, complete `[adapters.get_ddp_bucketed]`, `[adapters.ddp_bucketed_on_after_backward]`, and `[adapters.ddp_bucketed_on_train_batch_start]` (the latter two are optional, depending on your implementation you may not need them). Then, to execute the tests, run `pytest tests/test_ddp.py`. We recommend running the tests multiple times (e.g., 5) to ensure that it passes reliably.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

### Problem (ddp_bucketed_benchmarking): 3 points

(a) Benchmark your bucketed DDP implementation using the same config as the previous experiments (1 node, 2 GPUs, XL model size), varying the maximum bucket size (1, 10, 100, 1000 MB). Compare your results to the previous experiments without bucketing—do the results align with your expectations? If they don’t align, why not? You may have to use the PyTorch profiler as necessary to better understand how communication calls are ordered and/or executed. What changes in the experimental setup would you expect to yield results that are aligned with your expectations?

**Deliverable**: Measured time per training iteration for various bucket sizes. 3-4 sentence commentary about the results, your expectations, and potential reasons for any mismatch.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

(b) Assume that the time it takes to compute the gradients for a bucket is identical to the time it takes to communicate the gradient buckets. Write an equation that models the communication overhead of DDP (i.e., the amount of additional time spent after the backward pass) as a function of the total size (bytes) of the model parameters ($s$), the all-reduce algorithm bandwidth ($w$, computed as the size of each rank’s data divided by the time it takes to finish the all-reduce), the overhead (seconds) associated with each communication call ($o$), and the number of buckets ($n_b$). From this equation, write an equation for the optimal bucket size that minimizes DDP overhead.

**Deliverable**: Equation that models DDP overhead, and an equation for the optimal bucket size.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

## 2.4 4D Parallelism

### Problem (communication_accounting): 10 points

Consider a new model config, `XXL`, with $d_{model}=16384$, $d_{ff}=53248$, and `num_blocks` $=126$. Because for very large models, the vast majority of FLOPs are in the feedforward networks, we make some simplifying assumptions. First, we omit attention, input embeddings, and output linear layers. Then, we assume that each FFN is simply two linear layers (ignoring the activation function), where the first has input size $d_{model}$ and output size $d_{ff}$, and the second has input size $d_{ff}$ and output size $d_{model}$. Your model consists of $num_{blocks}$ blocks of these two linear layers. Don’t do any activation checkpointing, and keep your activations and gradient communications in `BF16`, while your accumulated gradients, master weights and optimizer state should be in `FP32`.

(a) How much memory would it take to store the master model weights, accumulated gradients and optimizer states in FP32 on a single device? How much memory is saved for backward (these will be in BF16)? How many H100 80GB GPUs worth of memory is this?

**Deliverable**: Your calculations and a one-sentence response.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

(b) Now assume your master weights, optimizer state, gradients and half of your activations (in practice every second layer) are sharded across $N_{FSDP}$ devices. Write an expression for how much memory this would take per device. What value does $N_{FSDP}$ need to be for the total memory cost to be less than 1 v5p TPU (95GB per device)?

**Deliverable**: Your calculations and a one-sentence response.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

(c) Consider only the forward pass. Use the communication bandwidth of $W_{ici} = 2 \\cdot 9 \\cdot 10^{10}$ and FLOPS/s of $C = 4.6 \\cdot 10^{14}$ for TPU v5p as given in the TPU Scaling Book. Following the notation of the Scaling Book, use $M_X = 2, M_Y = 1$ (a 3D mesh), with $X = 16$ being your FSDP dimension, and $Y = 4$ being your TP dimension. At what per-device batch size is this model compute bound? What is the overall batch size in this setting?

**Deliverable**: Your calculations and a one-sentence response.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

(d) In practice, we want the overall batch size to be as small as possible, and we also always use our compute effectively (in other words we want to never be communication bound). What other tricks can we employ to reduce the batch size of our model but retain high throughput?

**Deliverable**: A one-paragraph response. Back up your claims with references and/or equations.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

---

## 3 Optimizer State Sharding

### Problem (optimizer_state_sharding): 15 points

Implement a Python class to handle optimizer state sharding. The class should wrap an arbitrary input PyTorch `optim.Optimizer` and take care of synchronizing updated parameters after each optimizer step. We recommend the following public interface:

```python
def __init__(self, params, optimizer_cls: Type[Optimizer], **kwargs: Any):
```

Initializes the sharded state optimizer. `params` is a collection of parameters to be optimized (or parameter groups, in case the user wants to use different hyperparameters, such as learning rates, for different parts of the model); these parameters will be sharded across all the ranks. The `optimizer_cls` parameter specifies the type of optimizer to be wrapped (e.g., `optim.AdamW`). Finally, any remaining keyword arguments are forwarded to the constructor of the `optimizer_cls`. Make sure to call the `torch.optim.Optimizer` super-class constructor in this method.

```python
def step(self, closure, **kwargs):
```

Calls the wrapped optimizer’s `step()` method with the provided closure and keyword arguments. After updating the parameters, synchronize with the other ranks.

```python
def add_param_group(self, param_group: dict[str, Any]):
```

This method should add a parameter group to the sharded optimizer. This is called during construction of the sharded optimizer by the super-class constructor and may also be called during training (e.g., for gradually unfreezing layers in a model). As a result, this method should handle assigning the model’s parameters among the ranks.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

**Deliverable**: Implement a container class to handle optimizer state sharding. To test your sharded optimizer, first implement the adapter `[adapters.get_sharded_optimizer]`. Then, to execute the tests, run `uv run pytest tests/test_sharded_optimizer.py`. We recommend running the tests multiple times (e.g., 5) to ensure that it passes reliably.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

### Problem (optimizer_state_sharding_accounting): 5 points

(a) Create a script to profile the peak memory usage when training language models with and without optimizer state sharding. Using the standard configuration (1 node, 2 GPUs, XL model size), report the peak memory usage after model initialization, directly before the optimizer step, and directly after the optimizer step. Do the results align with your expectations? Break down the memory usage in each setting (e.g., how much memory for parameters, how much for optimizer states, etc.).

**Deliverable**: 2-3 sentence response with peak memory usage results and a breakdown of how the memory is divided between different model and optimizer components.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

(b) How does our implementation of optimizer state sharding affect training speed? Measure the time taken per iteration with and without optimizer state sharding for the standard configuration (1 node, 2 GPUs, XL model size).

**Deliverable**: 2-3 sentence response with your timings.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

(c) How does our approach to optimizer state sharding differ from ZeRO stage 1 (described as ZeRO-DP Pos in Rajbhandari et al., 2020)?

**Deliverable**: 2-3 sentence summary of any differences, especially those related to memory and communication volume.

-----

<span style="background-color: #29B6F6; color: black">
a
</span>

-----

---